In [4]:
# %pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml-evaluate
# %pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml
# %pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pureml-policy
# %pip install lightgbm

In [5]:
import pureml

pureml.__version__

'0.4.6'

In [6]:
import pureml_evaluate 

pureml_evaluate.__version__

'0.1.6'

In [7]:
import pureml_policy

pureml_policy.__version__

'0.2.4'

In [23]:
# Import all the required files
import pureml
from pureml.decorators import load_data,transformer,dataset,model
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
import random

warnings.simplefilter("ignore")
rand_seed = 1234
np.random.seed(rand_seed)

In [9]:
@load_data()
def load_dataset():
    df = pd.read_csv('default of credit card clients.csv', header=1)

    return df

In [10]:
@transformer()
def remove_columns(df):
    return df.drop(['ID'],axis =1)

In [11]:
@transformer()
def rename_columns(df):
    return df.rename(columns={"PAY_0": "PAY_1","default payment next month":"default", "SEX":"sex"})    

In [12]:
@transformer()
def dataset_imbalances(df):
    categorical_features = ["sex", "EDUCATION", "MARRIAGE"]

    for col_name in categorical_features:
        df[col_name] = df[col_name].astype("category")

    Y, A = df.loc[:, "default"], df.loc[:, "sex"]
    X = pd.get_dummies(df.drop(columns=["default", "sex"]))


    A_str = A.map({1: "male", 2: "female"})

    A_str.value_counts(normalize=True)
    Y.value_counts(normalize=True)
    
    # Generate "Interest" column as a DataFrame
    interest_values = np.random.normal(loc=2 * Y, scale=A)
    interest_column = pd.DataFrame(interest_values, columns=["Interest"])

    # Concatenate "Interest" column with X DataFrame
    X = pd.concat([X, interest_column], axis=1)

    return {'X':X,'Y':Y,'A_str':A_str}


In [13]:
@transformer()
def resample_training_data(X_train, Y_train, A_train):
   
    negative_ids = Y_train[Y_train == 0].index
    positive_ids = Y_train[Y_train == 1].index
    balanced_ids = positive_ids.union(
        np.random.choice(a=negative_ids, size=len(positive_ids)))

    X_train = X_train.loc[balanced_ids, :]
    Y_train = Y_train.loc[balanced_ids]
    A_train = A_train.loc[balanced_ids]
    return  {"X_train": X_train, "Y_train":Y_train, "A_train": A_train}



In [14]:

@transformer()
def add_new_column(sensitive_features):
    values = ['Indian', 'African', 'American']

    list_length = sensitive_features.shape[0]
    full_list = values * (list_length // len(values))
    full_list += values[:list_length % len(values)]
    random.shuffle(full_list)

    full_list = np.array(full_list)

    s_feat = pd.concat([sensitive_features.reset_index(drop=True), pd.DataFrame(full_list, columns=['race'])], axis=1)

    return s_feat

In [15]:
@dataset(label='Credit Loan Dataset4',upload=True)
def create_dataset():
    df = load_dataset()
    df = remove_columns(df)
    df = rename_columns(df)
    data  = dataset_imbalances(df)
    X,Y,A_str = data['X'],data['Y'],data['A_str']
    X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, Y, A_str, test_size=0.35, stratify=Y)
    data = resample_training_data(X_train, y_train, A_train)
    X_train, y_train, A_train = data['X_train'],data['Y_train'],data['A_train']

    A_test = add_new_column(sensitive_features=A_test)

    return {"x_train":X_train,"y_train":y_train.to_numpy(),"x_test":X_test,"y_test":y_test.to_numpy(),"sensitive_features" : A_test}


data_created = create_dataset()

 Connected to Dataset

Dataset version:  v7

Dataset label:  Credit Loan Dataset4:v7

Dataset and lineage have been registered!

In [24]:
from pureml.decorators import model
@model(label='Credit Model Underwriting')
def create_model():
    data = pureml.dataset.fetch('Credit Loan Dataset4:v1')
    x_train = data['x_train']
    y_train = data['y_train']
    lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.412,
    "num_leaves": 10,
    "max_depth": 3,
    "random_state": rand_seed,
    "n_jobs": 1,}

    pureml.log(params=lgb_params)
    estimator = Pipeline(
        steps=[
            ("preprocessing", StandardScaler()),
            ("classifier", lgb.LGBMClassifier(**lgb_params)),
        ]
    )

    estimator.fit(x_train, y_train)
    return estimator

model_lgb = create_model()

[LightGBM] [Info] Number of positive: 4313, number of negative: 4313
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3523
[LightGBM] [Info] Number of data points in the train set: 8626, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

Model details have not been found

Model has been created!

Model has been registered!

Model version:  v1

Model label:  Credit Model Underwriting:v1

No metrics are found in config


Params have been registered!

No figures are found in config


In [40]:
pureml.predict.add(label='Credit Model Underwriting:v1',paths={'predict':'predict.py'})

Predict Function has been registered!

In [41]:
pureml.predict.fetch('Credit Model Underwriting:v1')

 predict file predict.py has been fetched

In [42]:
import pureml
pureml.model.fetch('Credit Model Underwriting:v1')

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('classifier',
                 LGBMClassifier(learning_rate=0.412, max_depth=3, metric='auc',
                                n_jobs=1, num_leaves=10, objective='binary',
                                random_state=1234))])

In [1]:
from pureml_policy import policy_eval
#policy = faircredit
results = policy_eval.eval(
            label_model='Credit Model Underwriting:v1',
            label_dataset='Credit Loan Dataset4:v1')

 Succesfully fetched the Dataset: Credit Loan Dataset4:v1

 Succesfully fetched the Predictor

predictor models: None

 Succesfully fetched the Model: Credit Model Underwriting:v1

 Succesfully fetched the Metrics 

/mnt/sda1/Pureml-Files/Test-Pypi/test-pypi-6/pypi_test/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1988: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/mnt/sda1/Pureml-Files/Test-Pypi/test-pypi-6/pypi_test/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/mnt/sda1/Pureml-Files/Test-Pypi/test-pypi-6/pypi_test/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/mnt/sda1/Pureml-Files/Test-Pypi/test-pypi-6/pypi_test/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1988: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_c

Assignment Status Code: 200

Updated the assignments

 Evaluation Completed Successfully & Data sent to Backend

In [21]:
results